In [ ]:
import re
import numpy as np
import pandas as pd

# chat to csv

- removes "image ommited" messages"
- removes some weird `\u200e` characters
- removes timestamps
- senders removed from message and moved to separated column

In [ ]:
removes = ["\u200e"]
preamble = r"^\[[0-9]*/[0-9]*/[0-9]*, [0-9]*:[0-9]*:[0-9]*\] "
negated_preamble = r"^(?!\[[0-9]*/[0-9]*/[0-9]*, [0-9]*:[0-9]*:[0-9]*\]).*"
martin_preamble = r"^\[[0-9]*/[0-9]*/[0-9]*, [0-9]*:[0-9]*:[0-9]*\] martin brisiak: "
patricia_preamble = r"^\[[0-9]*/[0-9]*/[0-9]*, [0-9]*:[0-9]*:[0-9]*\] Patricia Svitková: "

with open("../_chat.txt") as file:
    lines = [line.rstrip() for line in file]
lines = lines[1:]
lines = [line.replace(remove, "") for line in lines for remove in removes]
lines = [line for line in lines if not line.endswith("image omitted")]
concated_lines = []
cache = []
for line in reversed(lines):
    if bool(re.match(preamble, line)) and bool(cache):
        concated_lines += [line + "\n" + "\n".join(reversed(cache))]
        cache = []
    elif bool(re.match(preamble, line)):
        concated_lines += [line]
    else:
        cache += [line]
concated_lines += cache
concated_lines = list(reversed(concated_lines))

df = pd.DataFrame(
    data=[
        [
            "martin" if bool(re.match(martin_preamble, line)) else "patricia",
            re.sub(martin_preamble, "", line) if bool(re.match(martin_preamble, line)) else re.sub(patricia_preamble, "", line)
        ]
        for line in concated_lines
    ],
    columns=["from", "message"]
)
df = df[(df["message"]!="This message was deleted.") & (df["message"]!="You deleted this message.") & (df["message"]!="video omitted")]
df.to_csv("./chat.csv")
df.head(100)

In [ ]:
df = pd.read_csv("./chat.csv")
text = "\n".join(df["message"].to_list())
len(text), text[3400:4500]